In [114]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import sidetable
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [115]:
df_da = pd.read_csv('data/df_da.csv', index_col = 'index')
pd.options.display.max_columns = None
df_da.sample(4)

,age,gender,country,education,role,prog_language,code_env,notebook_kaggle,notebook_colab,notebook_azure,notebook_paperspace,notebook_binder,notebook_codeocean,notebook_ibmwatsonstudio,notebook_amazonsagemaker,notebook_amazonemr,notebook_googlecloudnotebooks,notebook_googleclouddatalab,notebook_databricks,notebook_zeppelin,notebook_deepnote,notebook_observable,notebook_OTHER,viz,field,salary,learn_dp_MySQL,learn_dp_PostgreSQL,learn_dp_SQLite,learn_dp_Oracle,learn_dp_Mongo,learn_dp_Snowflake,learn_dp_IBMDd2,learn_dp_SQLServer,learn_dp_AzureSQL,learn_dp_AzureCosmos,learn_dp_Redshift,learn_dp_Aurora,learn_dp_RDS,learn_dp_DynamoDB,learn_dp_BigQuery,learn_dp_CloudSQL,learn_dp_Firestore,learn_dp_BigTable,learn_dp_Spanner,learn_dp_Other,dp_most_freq,BI_tools,share_plotlydash,share_streamlit,share_NBViewer,share_GitHub,share_Personal blog,share_Kaggle,share_Colab,share_Shiny,share_None,share_OTHER,courses_Coursera,courses_edX,courses_Kaggle,courses_DataCamp,courses_Fastai,courses_Udacity,courses_Udemy,courses_LinkedIn,courses_Cloud-certification,courses_University,courses_OTHER,media_Twitter,media_Email,media_Reddit,media_Kaggle,media_Forums,media_YouTube,media_Podcasts,media_Blogs,media_Journal,media_Slack,media_OTHER,cat_experience,cat_salary
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
15894,30-34,Man,Chile,Master’s degree,Data Analyst,"SQL, Other",Notepad++,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Ggplot / ggplot2 ,Geoplotlib",Retail/Sales,"20,000-24,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MySQL,Qlik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I do not share my work publicly,NaN,Coursera,edX,Kaggle Learn Courses,DataCamp,NaN,Udacity,Udemy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Kaggle (notebooks, forums, etc)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Junior,< 25K
4382,18-21,Woman,India,Master’s degree,Business Analyst,"Python, SQL","RStudio ,Visual Studio Code (VSCode) ,Jupyter ...",Kaggle Notebooks,Colab Notebooks,NaN,NaN,Binder / JupyterHub,NaN,NaN,Amazon Sagemaker Studio Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express ...",Computers/Technology,"1,000-1,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MySQL,Other,NaN,NaN,NaN,GitHub,Personal blog,NaN,NaN,NaN,NaN,NaN,Coursera,NaN,NaN,DataCamp,NaN,Udacity,NaN,NaN,NaN,University Courses (resulting in a university ...,Other,Twitter (data science influencers),"Email newsletters (Data Elixir, O'Reilly Data ...",NaN,NaN,"Course Forums (forums.fast.ai, Coursera forums...","YouTube (Kaggle YouTube, Cloud AI Adventures, ...",NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,Junior,< 25K
6596,50-54,Man,Japan,Master’s degree,Business Analyst,Other,RStudio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Insurance/Risk Assessment,"10,000-14,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tableau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I do not share my work publicly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Senior+,< 25K
20744,18-21,Man,India,Bachelor’s degree,Data Analyst,"Python, Java","Visual Studio ,Visual Studio Code (VSCode)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Junior,NaN


In [116]:
df_da.describe(include='object').T

,count,unique,top,freq
age,3269,11,25-29,788
gender,3269,5,Man,2538
country,3269,66,India,795
education,3269,7,Master’s degree,1514
role,3269,2,Data Analyst,2301
...,...,...,...,...
media_Journal,260,1,"Journal Publications (peer-reviewed journals, ...",260
media_Slack,258,1,"Slack Communities (ods.ai, kagglenoobs, etc)",258
media_OTHER,82,1,Other,82
cat_experience,3269,5,Junior,1881


In [117]:
df_da.stb.missing()

,missing,total,percent
notebook_codeocean,3247,3269,99.327011
notebook_paperspace,3246,3269,99.296421
notebook_deepnote,3243,3269,99.204650
notebook_zeppelin,3241,3269,99.143469
share_NBViewer,3241,3269,99.143469
...,...,...,...
role,0,3269,0.000000
education,0,3269,0.000000
country,0,3269,0.000000
cat_experience,0,3269,0.000000


In [118]:
# creamos diferentes df con datos sociodemográficos propios para cada columna que nos interesa 
# y poder expandirlas sin crear demasiadas filas


sociodemo_language = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","prog_language"] ]
sociodemo_language["languages"] = sociodemo_language["prog_language"].str.split(",")
df_languages = sociodemo_language.explode(["languages"])
df_languages.drop("prog_language", inplace= True, axis= 1)
df_languages.sample(2)

/tmp/ipykernel_15691/4217408243.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sociodemo_language["languages"] = sociodemo_language["prog_language"].str.split(",")


,age,gender,country,education,role,field,cat_experience,cat_salary,languages
index,,,,,,,,,
14031,18-21,Man,Indonesia,I prefer not to answer,Data Analyst,Academics/Education,Junior,NaN,SQL
12828,30-34,Man,Viet Nam,Professional doctorate,Data Analyst,Academics/Education,Junior,< 25K,Python


In [119]:
sociodemo_code_env = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","code_env"] ]
sociodemo_code_env["cod_env"] = sociodemo_code_env["code_env"].str.split(",")
df_code_env = sociodemo_code_env.explode(["cod_env"])
df_code_env.drop("code_env", inplace= True, axis= 1)
df_code_env.sample(2)

/tmp/ipykernel_15691/993416549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sociodemo_code_env["cod_env"] = sociodemo_code_env["code_env"].str.split(",")


,age,gender,country,education,role,field,cat_experience,cat_salary,cod_env
index,,,,,,,,,
17928,25-29,Man,India,Master’s degree,Business Analyst,Computers/Technology,Junior,< 25K,Jupyter Notebook
20403,25-29,Man,Bangladesh,Bachelor’s degree,Data Analyst,Online Service/Internet-based Services,Junior,< 25K,Visual Studio Code (VSCode)


In [120]:
sociodemo_viz = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","viz"] ]
sociodemo_viz["visual"] = sociodemo_viz["viz"].str.split(",")
df_viz = sociodemo_viz.explode(["visual"])
df_viz.drop("viz", inplace= True, axis= 1)
df_viz.sample(2)

/tmp/ipykernel_15691/2257016201.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sociodemo_viz["visual"] = sociodemo_viz["viz"].str.split(",")


,age,gender,country,education,role,field,cat_experience,cat_salary,visual
index,,,,,,,,,
22052,30-34,Man,India,Master’s degree,Business Analyst,Computers/Technology,Mid,25K - 50K,Seaborn
23045,40-44,Man,Brazil,Bachelor’s degree,Business Analyst,Computers/Technology,Senior+,< 25K,None


In [121]:
#Creamos la función para crear una columna que una las anteriores:
def unir(col):
    lista_valores = []
    for element in col:
        if str(element) != "nan":
            lista_valores.append(str(element))
        else:
            pass
    return lista_valores

In [122]:
# creamos una lista con todas las columnas relacionadas a la pregunta notebooks

lista_notebooks = ['notebook_kaggle', 'notebook_colab', 'notebook_azure',
       'notebook_paperspace', 'notebook_binder', 'notebook_codeocean',
       'notebook_ibmwatsonstudio', 'notebook_amazonsagemaker',
       'notebook_amazonemr', 'notebook_googlecloudnotebooks',
       'notebook_googleclouddatalab', 'notebook_databricks',
       'notebook_zeppelin', 'notebook_deepnote', 'notebook_observable',
       'notebook_OTHER']

In [123]:
# aplicamos la función de unir columnas sobre el df completo pero uniendo solo las columnas 'notebooks':
df_da['notebooks_unido'] = df_da[lista_notebooks].apply(unir, axis=1)

In [124]:
#comprobamos:
df_da.head(2)

,age,gender,country,education,role,prog_language,code_env,notebook_kaggle,notebook_colab,notebook_azure,notebook_paperspace,notebook_binder,notebook_codeocean,notebook_ibmwatsonstudio,notebook_amazonsagemaker,notebook_amazonemr,notebook_googlecloudnotebooks,notebook_googleclouddatalab,notebook_databricks,notebook_zeppelin,notebook_deepnote,notebook_observable,notebook_OTHER,viz,field,salary,learn_dp_MySQL,learn_dp_PostgreSQL,learn_dp_SQLite,learn_dp_Oracle,learn_dp_Mongo,learn_dp_Snowflake,learn_dp_IBMDd2,learn_dp_SQLServer,learn_dp_AzureSQL,learn_dp_AzureCosmos,learn_dp_Redshift,learn_dp_Aurora,learn_dp_RDS,learn_dp_DynamoDB,learn_dp_BigQuery,learn_dp_CloudSQL,learn_dp_Firestore,learn_dp_BigTable,learn_dp_Spanner,learn_dp_Other,dp_most_freq,BI_tools,share_plotlydash,share_streamlit,share_NBViewer,share_GitHub,share_Personal blog,share_Kaggle,share_Colab,share_Shiny,share_None,share_OTHER,courses_Coursera,courses_edX,courses_Kaggle,courses_DataCamp,courses_Fastai,courses_Udacity,courses_Udemy,courses_LinkedIn,courses_Cloud-certification,courses_University,courses_OTHER,media_Twitter,media_Email,media_Reddit,media_Kaggle,media_Forums,media_YouTube,media_Podcasts,media_Blogs,media_Journal,media_Slack,media_OTHER,cat_experience,cat_salary,notebooks_unido
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,"Python, SQL","Jupyter (JupyterLab, Jupyter Notebooks, etc) ,...",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Databricks Collaborative Notebooks,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express",Energy/Mining,"2,000-2,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Microsoft Azure SQL Database,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coursera,NaN,Kaggle Learn Courses,NaN,NaN,NaN,NaN,NaN,"Cloud-certification programs (direct from AWS,...",NaN,NaN,NaN,NaN,NaN,"Kaggle (notebooks, forums, etc)",NaN,NaN,NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,Senior+,< 25K,"[ Kaggle Notebooks, Colab Notebooks, Databric..."
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,R,RStudio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Cloud Datalab,NaN,NaN,NaN,NaN,NaN,Ggplot / ggplot2,Manufacturing/Fabrication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Junior,NaN,[Google Cloud Datalab]


In [125]:
# Nos vamos a quedar solo con las columnas sociodemograficas y la nueva creada:

sociodemo_notebooks = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","notebooks_unido"]]
df_notebooks = sociodemo_notebooks.explode(["notebooks_unido"])
df_notebooks.head()

,age,gender,country,education,role,field,cat_experience,cat_salary,notebooks_unido
index,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,Kaggle Notebooks
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,Colab Notebooks
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,Databricks Collaborative Notebooks
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,Manufacturing/Fabrication,Junior,NaN,Google Cloud Datalab
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,Kaggle Notebooks


In [126]:
# vamos a comprobar si quedan nulos en esta columna (muestra que no haya usado nunca uno de estos notebooks):

df_notebooks['notebooks_unido'].isnull().sum()

1219

In [127]:
# vamos a eliminar estas filas (solo queremos saber los notebooks más utilizados):

df_notebooks.dropna(subset= 'notebooks_unido', axis = 0, inplace= True)

In [128]:
df_notebooks['notebooks_unido'].isnull().sum()

0

In [129]:
# vamos a hacer lo mismo con las columnas learn:

col_lear_dp = ['learn_dp_MySQL', 'learn_dp_PostgreSQL', 'learn_dp_SQLite', 'learn_dp_Oracle', 'learn_dp_Mongo', 'learn_dp_Snowflake', 'learn_dp_IBMDd2', 'learn_dp_SQLServer',
'learn_dp_AzureSQL', 'learn_dp_AzureCosmos', 'learn_dp_Redshift', 'learn_dp_Aurora',
'learn_dp_RDS', 'learn_dp_DynamoDB', 'learn_dp_BigQuery', 'learn_dp_CloudSQL', 'learn_dp_Firestore', 'learn_dp_BigTable', 'learn_dp_Spanner', 'learn_dp_Other']

In [130]:
df_da['learn_unido'] = df_da[col_lear_dp].apply(unir, axis=1)

In [131]:
sociodemo_learn = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","learn_unido"]]
df_learn = sociodemo_learn.explode(["learn_unido"])
df_learn.head()

,age,gender,country,education,role,field,cat_experience,cat_salary,learn_unido
index,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,Microsoft Azure SQL Database
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,Manufacturing/Fabrication,Junior,NaN,NaN
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,MySQL
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,SQLite
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,MongoDB


In [132]:
df_learn['learn_unido'].isnull().sum()

2134

In [133]:
df_learn.dropna(subset= 'learn_unido', axis = 0, inplace= True)

In [134]:
df_learn['learn_unido'].isnull().sum()

0

In [135]:
col_share = ['share_plotlydash', 'share_streamlit', 'share_NBViewer', 'share_GitHub', 'share_Personal blog', 'share_Kaggle', 'share_Colab', 'share_Shiny', 'share_None', 'share_OTHER']

In [136]:
df_da['share_unido'] = df_da[col_share].apply(unir, axis=1)

In [137]:
sociodemo_share = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","share_unido"]]
df_share = sociodemo_share.explode(["share_unido"])
df_share.head()

,age,gender,country,education,role,field,cat_experience,cat_salary,share_unido
index,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,NaN
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,Manufacturing/Fabrication,Junior,NaN,NaN
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,I do not share my work publicly
46,22-24,Man,China,Some college/university study without earning ...,Data Analyst,Computers/Technology,Junior,< 25K,NaN
52,40-44,Man,South Africa,Master’s degree,Data Analyst,Manufacturing/Fabrication,Junior,25K - 50K,I do not share my work publicly


In [138]:
df_share['share_unido'].isnull().sum()

1610

In [139]:
df_share.dropna(subset= 'share_unido', axis = 0, inplace= True)

In [140]:
df_share['share_unido'].isnull().sum()

0

In [141]:
col_courses = ['courses_Coursera', 'courses_edX', 'courses_Kaggle', 'courses_DataCamp', 'courses_Fastai', 'courses_Udacity', 'courses_Udemy', 'courses_LinkedIn',
'courses_Cloud-certification', 'courses_University', 'courses_OTHER']

In [142]:
df_da['courses_unido'] = df_da[col_courses].apply(unir, axis=1)

In [143]:
sociodemo_courses = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","courses_unido"]]
df_courses = sociodemo_courses.explode(["courses_unido"])
df_courses.head()

,age,gender,country,education,role,field,cat_experience,cat_salary,courses_unido
index,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,Coursera
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,Kaggle Learn Courses
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,"Cloud-certification programs (direct from AWS,..."
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,Manufacturing/Fabrication,Junior,NaN,NaN
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,Coursera


In [144]:
df_courses['courses_unido'].isnull().sum()

889

In [145]:
df_courses.dropna(subset= 'courses_unido', axis = 0, inplace= True)

In [146]:
df_courses['courses_unido'].isnull().sum()

0

In [147]:
col_media = ['media_Twitter', 'media_Email', 'media_Reddit', 'media_Kaggle', 'media_Forums', 'media_YouTube', 'media_Podcasts', 'media_Blogs', 'media_Journal', 'media_Slack', 'media_OTHER']

In [148]:
df_da['media_unido'] = df_da[col_media].apply(unir, axis=1)

In [149]:
sociodemo_media = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","media_unido"]]
df_media = sociodemo_media.explode(["media_unido"])
df_media.head()

,age,gender,country,education,role,field,cat_experience,cat_salary,media_unido
index,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,"Kaggle (notebooks, forums, etc)"
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,"Blogs (Towards Data Science, Analytics Vidhya,..."
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,Manufacturing/Fabrication,Junior,NaN,NaN
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,Twitter (data science influencers)
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,"Email newsletters (Data Elixir, O'Reilly Data ..."


In [150]:
df_media['media_unido'].isnull().sum()

942

In [151]:
df_media.dropna(subset= 'media_unido', axis = 0, inplace= True)

In [152]:
df_media['media_unido'].isnull().sum()

0

In [153]:
df_languages.to_csv("data/df_languages.csv")
df_viz.to_csv("data/df_viz.csv")
df_code_env.to_csv("data/df_code_env.csv")
df_notebooks.to_csv("data/df_notebooks.csv")
df_learn.to_csv("data/df_learn.csv")
df_share.to_csv("data/df_share.csv")
df_courses.to_csv("data/df_courses.csv")
df_media.to_csv("data/df_media.csv")